# Supercontinuum and second harmonic generations

## Single Envelope

Method from [1]. I plan to validate according to their first example, quoting:

"We solved Eq. (11) by a split-step Fourier method exploiting the fourth-order Runge-Kutta scheme for the nonlinear step.
In order to show the validity of our equation, we simulated the propagation of a femtosecond pulse in a $L = 5$ mm long periodically poled lithium tantalate sample (PPLT). To model the refractive index dispersion we employed a Sellmeier model fitted from experimental data [13] and a nonlinear coefficient of $d_{33} = \chi^{(2)}_{LT}/2 = 10.6$ pm/V. In the numerical code wei nserted the exact dispersion relation $k(ω)$. We assumed a first-order quasi-phase-matching (QPM) grating, with a period $\Lambda = 17.4$ μm. We thus allowed a periodic variation of the nonlinear coefficient $\chi^{(2)} = \chi^{(2)}(z) = 2/\pi \chi^{(2)}_{LT} e^{i2π/\Lambda z} + c.c.$ We injected a $T = 60$ fs FWHM long Gaussian pulse, centered around 1400 nm, with I = 10 GW/cm2 peak intensity. The corresponding residual phase mismatch is $k = 2k(ωin) − k(2ωin) = 10 000$ m$^{−1}$, where $\omega_{in}$ is the carrier frequency of the input pulse. In the simulation we set the reference frequency $\omega_0$ to be equal to the second harmonic of the input pulse: in this way the second harmonic is stationary in the reference frame $(z,τ)$."

[1]M. Conforti, F. Baronio, and C. De Angelis, “Nonlinear envelope equation for broadband optical pulses in quadratic media,” Phys. Rev. A, vol. 81, no. 5, p. 053841, May 2010, doi: 10.1103/PhysRevA.81.053841.

We begin by the typical Python imports.

In [ ]:
import numpy as np
from numpy.fft import fft, ifft, fftshift, fftfreq
import matplotlib.pyplot as plt
import time
from scipy import fftpack as sp
from scipy import signal
import copy
from matplotlib import cm
import colorcet as cc
from matplotlib.colors import Normalize

#This are my libraries
import nlo 
import materials
import waveguides
from util import sech

## Units
We'll work in SI units. Mostly in base units.

In [ ]:
from scipy.constants import pi, c, epsilon_0

## Time and Frequency domain windows

Now we need to determine an FFT size $N$, this will be the common size for the time and frequency axis in our simulations. We also need to select how large the time window $2 T_{\mathrm{max}}$ for our simulations will be. The time window needs to be large enough so that pulses don't reach the edge of the domain at any point. Since we'll usually pick a reference frame that moves with one of the pulses, then how much the other pulses travel depends on the respective group velocity mismatch. The sampling period $\Delta t$ will be determined by the size of the window and the number of points; Nyquist criteria must be satisfied as usual.

Once the number of points and time-window has been selected, the frequency grid is fixed by the FFT, with a frequency resolution
$$ F_s = \frac{1}{N \Delta t},$$
the minimum relative frequency is 0 while the maximum is
$$ BW = \frac{1}{\Delta t}.$$
The maximum relative frequency should alway be below the desired absolute carrier frequency. So the longest wavelength allowed becomes:
$$ \lambda_{\mathrm{max}} = \frac{c}{F_{\mathrm{max}}} = 2c\Delta t $$

The reference frequency $f_{ref}$ can be chosen arbitrarely, but without more information a good start point could be the geometric mean between the min and max frequencies of interest, shifted by fmin.

In [ ]:
wl_max = 2.5e-6
wl_min = 0.3e-6
f_max = c/wl_min
f_min = c/wl_max
f_ref = 400e12

omega_ref = 2*pi*f_ref
wl_ref = c/f_ref
BW = f_max-f_min #This is the bandwidth desired
dt = 1/BW
NFFT = 2**13 #Number of FFT points
Twindow = dt*NFFT
print('Reference wavelength = %0.1f nm' %(wl_ref*1e9))
print('Reference frequency = %0.1f THz' %(f_ref*1e-12))
print('Min absolute frequency needed = %0.1f THz' %(f_min*1e-12))
print('Max absolute frequency needed = %0.1f THz' %(f_max*1e-12))
print('Bandwidth needed = %0.1f THz' %(BW*1e-12))
print('Time resolution needed = %0.3f fs' %(dt*1e15))
print('Maximum size of time window based on %i FFT points = %0.3f fs' %(NFFT, Twindow*1e15))
print('\n')


Twindow = 12e-12
t = np.linspace(-Twindow/2, Twindow/2, NFFT)
dt = t[1]-t[0] #Sampling period
f = fftfreq(NFFT, dt)
omega = 2*pi*f
df = f[1]-f[0]
print('Actual time window chosen = %0.3f fs' %(Twindow*1e15))
print('Actual Sampling Rate = %0.3f fs' %(dt*1e15))
print('Frequency Resolution = %0.3f THz' %(df*1e-12))
print('Minimum absolute frequency = %0.3f THz' %((np.min(f) + f_ref)*1e-12))
print('Maximum absolute frequency = %0.3f THz' %((np.max(f) + f_ref)*1e-12))

## Input pulse creation

In [ ]:
#Parameters
# wl0 = 1.55e-6 #center wavelength
# f0 = c/wl0
f0 = 200e12
wl0 = c/f0
delta_omega = 2*pi*(f0 - f_ref)

#get the f0 frequency index
f0_index = np.abs(f+f_ref - f0).argmin()

# pulse_energy = 100e-12
pulse_energy = 100e-12

tau = 60e-15 #Pulse width

#Pulse creation
frep = 10e9
pulse = np.sqrt(0.939*pulse_energy/tau)*np.exp(-2*np.log(2)*(t/tau)**2)*np.exp(1j*delta_omega*t)
noise = 1e-20*np.random.normal(size=NFFT)
pulse = pulse + noise
pulse_energy = nlo.energy_td(t, pulse) #To check that everything went well

print('Center Wavelength = %0.3f um' %(wl0*1e6))
print('Center frequency = %0.3f THz' %(f0*1e-12))
print('FWHM = %0.3f fs' %(nlo.FWHM(t, pulse)*1e15))
print('Pulse Energy = %0.3f pJ' %(pulse_energy*1e12))
print('Pulse peak power = %0.3f W' %(pulse_energy/tau))
print('Comb average power = %0.3f mW' %(pulse_energy*frep*1e3))

plt.rcParams['figure.figsize'] = [15, 10]
fig = plt.figure()
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)
nlo.plot_magsq(t*1e15, pulse, ax=ax1);
nlo.plot_ESD_dB(t*1e12, pulse, ax=ax2)
nlo.plot_ESD_dB_absfreq(t*1e12, pulse, f_ref*1e-12, ax=ax3)
nlo.plot_ESD_dB_vs_wavelength(t, pulse, f_ref, ax=ax4)

ax2.scatter(f[f0_index]*1e-12, 0, c='k');
ax3.scatter((f[f0_index]+f_ref)*1e-12, 0, c='k');
ax4.scatter(c/(f[f0_index]+f_ref)*1e6, 0, c='k');

## Waveguides

In [ ]:
def get_waveguide_dispersion():
    #Waveguide geometry
    etch = 0.35
    hridge = 0.7
    hslab = hridge - etch
    wridge = 1.5
    
    #Effective index calculation
    neff = np.zeros(f.shape)
    for kf in range(f.size):
        freq = f[kf] + f_ref
        wl = c/freq*1e6 #Wavelength in microns

        #Material properties (frequency dependent)
        n0 = 1
        nLN = materials.refractive_index('LN_MgO_e', wl)
        nSiO2 = materials.refractive_index('SiO2', wl)

        neff[kf] = waveguides.neff_ridge(wl, nridge=nLN, nbox=nSiO2, nclad=n0, w=wridge, h=hridge, hslab=hslab)
    
    return neff

neff = get_waveguide_dispersion()

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]
plt.plot(fftshift(f+f_ref)*1e-12, fftshift(neff));
plt.scatter((f[0]+f_ref)*1e-12, neff[0], c='k');
plt.scatter((f[f0_index]+f_ref)*1e-12, neff[f0_index], c='r');
plt.xlabel('Fourier Frequency (THz)');
plt.ylabel('Effective Index');
plt.grid(True)

The higher order dispersion operator is
$$D = \beta(\Omega) - \beta_0(\Omega=0) - \beta_1(\Omega=0) \Omega$$.

In [ ]:
beta = (omega+omega_ref)*neff/c
beta_1 = np.gradient(beta, 2*pi*df)
beta_2 = np.gradient(beta_1, 2*pi*df)

beta_ref = beta[0]
# beta_1_ref = beta_1[f0_index]
beta_1_ref = beta_1[0]
D = beta - beta_ref - beta_1_ref*omega

plt.rcParams['figure.figsize'] = [10, 5]
plt.plot(fftshift(f)*1e-12, fftshift(D));
plt.xlabel('Fourier Frequency (THz)');
plt.grid(True)
plt.ylabel('Higher order dispersion');

#Some extra information
Ld = (tau)**2/(4*np.log(2)*abs(beta_2))
GVM = 1/beta_1[f0_index] - 1/beta_1_ref

print('GVD at %0.3f um = %0.3f fs^2/mm' %(wl_ref*1e6, beta_2[0]*1e27))
print('GVD at %0.3f um = %0.3f fs^2/mm' %(wl0*1e6, beta_2[f0_index]*1e27))
print('Dispersion Length for Gaussian pulse at %0.3f um = %0.3f mm' %(wl_ref*1e6, Ld[0]*1e3))
print('Dispersion Length for Gaussian pulse at %0.3f um = %0.3f mm' %(wl0*1e6, Ld[f0_index]*1e3))
print('Group velocity of %0.3f um wave in moving reference plane = %0.3f mm/ps' %(wl0*1e6, GVM*1e3*1e-12))

Poling period for SHG:
$$
\Lambda = \frac{\lambda_{2\omega}}{n_{2\omega} - n_\omega}
$$

In [ ]:
n2w = neff[np.abs(f+f_ref - 2*f0).argmin()]
nw = neff[np.abs(f+f_ref - f0).argmin()]
pp = (wl0/2)/(n2w-nw)
print('Poling period for SHG starting at %0.3f um is %0.3f um' %(wl0*1e6, pp*1e6))

## Nonlinear  Stuff

In [ ]:
def poling_rectangular(z, pp):
    return np.sign(np.cos(z*2*pi/pp))

def poling_sinusoidal(z, pp):
    return np.cos(z*2*pi/pp)

def chi2(z):
    poling = poling_rectangular(z, pp)
#     poling = 1
    return 2*d33*poling

def chi(z):
    return chi2(z)*omega_ref/(2*neff[0]*c)

def nonlinear_operator(z, A):
    phi = omega_ref*t - (beta_ref - beta_1_ref*omega_ref)*z
    f1 = A*A*np.exp(1j*phi) + 2*A*np.conj(A)*np.exp(-1j*phi)
    f1_deriv = np.gradient(f1, dt)
    f = -1j*chi(z)*f1 - 1*(chi(z)/omega_ref)*f1_deriv
    return f

In [ ]:
def propagate(A, D):
    
    A_evol = 1j*np.zeros([t.size, Nsteps+1])
    A_evol[:,0] = A #Initial value
    
    Da = np.exp(-1j*D*h)
    z = 0
    
    for kz in range(Nsteps):
        #Linear step
        A = ifft(Da*fft(A))

        #Nonlinear step
        #Runge-Kutta 4th order
        k1 = nonlinear_operator(z    , A       )
        k2 = nonlinear_operator(z+h/2, A+h*k1/2)
        k3 = nonlinear_operator(z+h/2, A+h*k2/2)
        k4 = nonlinear_operator(z+h  , A+h*k3  )

        z = z + h
        A = A + (h/6)*(k1+2*k2+2*k3+k4) 
        
        A_evol[:, kz+1] = A
    
    return A, A_evol

In [ ]:
#Crystal parameters
L = 3e-3
h = 1e-3/50
Nsteps = int(L/h)
d33 = 1e-7
pp = pp

print('Crystal length = %0.2f mm' %(L*1e3))
print('Step size = %0.2f um' %(h*1e6))
print('Number of steps = %i' %(Nsteps))
print('Nonlinear coefficient, d33 = %0.2f pm/V' %(d33*1e12))
print('Using periodic poling with period = %0.3f um' %(pp*1e6))

## Single pass

In [ ]:
[a, a_evol] = propagate(pulse, D)
ax1 = nlo.plot_magsq(t, a)
ax1 = nlo.plot_magsq(t, pulse, ax=ax1)

In [ ]:
X,Y = np.meshgrid(t*1e15, np.arange(Nsteps+1)*h*1e3)
plt.pcolormesh(X, Y, (np.transpose(np.abs(a_evol))))
plt.xlabel('Time (fs)')
plt.ylabel('Propagation distance (mm)')
plt.colorbar()

In [ ]:
plt.pcolormesh(X, Y, (np.log10(np.transpose(np.abs(a_evol)))))
plt.xlabel('Time (fs)')
plt.ylabel('Propagation distance (mm)')
plt.colorbar()

In [ ]:
A_evol = np.zeros((t.size, Nsteps+1))

for k in range(Nsteps+1):
    x = a_evol[:,k]
    _ , Aesd = nlo.get_esd(t*1e15, x)
    A_evol[:,k] = 10*np.log10(Aesd)

A_evol = A_evol - np.amax(A_evol)
A_evol = fftshift(A_evol, axes=0)

X,Y = np.meshgrid(fftshift(f+f_ref)*1e-12, np.arange(Nsteps+1)*h*1e3)
plt.figure()
plt.pcolormesh(X, Y, (np.transpose(A_evol)), cmap = cc.cm["fire"], vmin=-80, vmax=0)
cb = plt.colorbar()
cb.set_label('Relative PSD (dB)')
plt.xlabel('Frequency THz')
plt.ylabel('Propagation distance (mm)');

wl_max = 2
wl_array = c/(fftshift(f)+f_ref)*1e6
wl_max_idx = np.abs(wl_array - wl_max).argmin()
wl_array = wl_array[wl_max_idx:]
Xwl,Ywl = np.meshgrid(wl_array, np.arange(Nsteps+1)*h*1e3)
plt.figure()
plt.pcolormesh(Xwl, Ywl, (np.transpose(A_evol[wl_max_idx:, :])), cmap = cc.cm["fire"], vmin=-80, vmax=0)
cb = plt.colorbar()
cb.set_label('Relative PSD (dB)')
plt.xlabel('Wavelength (um)')
plt.ylabel('Propagation distance (mm)');

## Filter and plot harmonics

In [ ]:
fs=1/dt*1e-12
sos = signal.butter(15, [0.45, 0.55], 'bandpass', output='sos')
w, hfilter = signal.sosfreqz(sos, worN=NFFT)

ax1 = nlo.plot_ESD_dB_absfreq(t*1e12, a, f_ref*1e-12)
ax1 = nlo.plot_ESD_dB_absfreq(t*1e12, pulse, f_ref*1e-12, ax=ax1)
ax1.plot(fftshift(f+f_ref)*1e-12, 20*np.log10(abs(hfilter)));
ax1.set_ylim([-100,10])

In [ ]:
filtered = ifft(fft(a)*fftshift(hfilter))
ax1 = nlo.plot_ESD_dB_absfreq(t*1e12, a, f_ref*1e-12)
ax1 = nlo.plot_ESD_dB_absfreq(t*1e12, pulse, f_ref*1e-12, ax=ax1)
ax1 = nlo.plot_ESD_dB_absfreq(t*1e12, filtered, f_ref*1e-12, ax=ax1)

In [ ]:
ax1 = nlo.plot_magsq(t*1e15, filtered)

In [ ]:
ax1 = nlo.plot_magsq(t*1e15, filtered)
ax1 = nlo.plot_magsq(t*1e15, a, ax=ax1)
ax1 = nlo.plot_magsq(t*1e15, pulse, ax=ax1, ylim=[0,100], xlim=[-2000,2000])